In [46]:
from sympy import *
from sympy.matrices import inverse
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, LagrangesMethod, Lagrangian
from sympy.physics import mechanics
init_vprinting()

import numpy as np
import math

In [47]:
def partial(arg, x):
    lst = []
    for v in x:
        lst.append(arg.diff(v))
    return Matrix([lst])

In [48]:
Is, Iw, Ib = symbols("I_s I_w I_b")
rs, rw, rtot, lam, ms, mw, mb, mtot, l = symbols("r_s r_w r_tot lambda m_s m_w m_b m_tot l")
t, g = symbols("t g")

psi, th = dynamicsymbols("psi theta")
T = dynamicsymbols("T")
dpsi = psi.diff(t)
dth = th.diff(t)

In [49]:
const_dict = {
    "m_s": 3.2,
    "m_ow": .555,
    "mm": .24,
    "r_s": 0.115,
    "h": 0.8,
    "I_s": 2.65e-2,
    "I_ow": 6.94e-2,
    "l": 0.405,
    "g": 9.81,
    "alpha": math.radians(45),
    
    "m_b": 7.135,
    "m_w": 0.995,
    "r_w": 0.050,
    "I_w": 1.90e-3,
    "I_b": 2.4
}
const_list = [(symbols(s), v) for s, v in const_dict.items()]
const_list = [("r_tot", rw + rs), (mtot, ms + mw + mb), (lam, mw*(rs+rw)+mb*l)] + const_list

In [59]:
q = Matrix([th, psi])
dq = q.diff(t)
x = Matrix([q, dq])
u = Matrix([T])

In [51]:
qbar_slist = [(th, 0), (psi, 0), (dth, 0), (dpsi, 0)]

In [52]:
M = Matrix([[Is + rs**2*mtot + rs**2/rw**2*Iw, rs*lam*cos(psi) - rs**2/rw**2*Iw], 
			[rs*lam*cos(psi) - rs**2/rw**2*Iw, rtot**2*mw + rs**2/rw**2*Iw+Ib]])
C = Matrix([[0, -rs*lam*dpsi*sin(psi)], [0, 0]])
G = Matrix([0, -lam*g*sin(psi)])

In [53]:
Text = Matrix([rs/rw, -rs/rw]) * T

In [54]:
Eq(M * dq.diff(t), Text)

⎡ ⎛        2              ⎞     ⎛           2           ⎞   ⎤           
⎢ ⎜  I_w⋅rₛ               ⎟     ⎜     I_w⋅rₛ           2⎟   ⎥           
⎢ ⎜- ─────── + λ⋅rₛ⋅cos(ψ)⎟⋅ψ̈ + ⎜Iₛ + ─────── + mₜₒₜ⋅rₛ ⎟⋅θ̈ ⎥   ⎡ rₛ⋅T ⎤
⎢ ⎜       2               ⎟     ⎜          2            ⎟   ⎥   ⎢ ──── ⎥
⎢ ⎝    r_w                ⎠     ⎝       r_w             ⎠   ⎥   ⎢ r_w  ⎥
⎢                                                           ⎥ = ⎢      ⎥
⎢⎛        2              ⎞     ⎛            2            ⎞  ⎥   ⎢-rₛ⋅T ⎥
⎢⎜  I_w⋅rₛ               ⎟     ⎜      I_w⋅rₛ            2⎟  ⎥   ⎢──────⎥
⎢⎜- ─────── + λ⋅rₛ⋅cos(ψ)⎟⋅θ̈ + ⎜I_b + ─────── + m_w⋅rₜₒₜ ⎟⋅ψ̈⎥   ⎣ r_w  ⎦
⎢⎜       2               ⎟     ⎜           2             ⎟  ⎥           
⎣⎝    r_w                ⎠     ⎝        r_w              ⎠  ⎦           

In [55]:
slist = const_list + qbar_slist
Minv = M.subs(slist).inv()
Minv

⎡ 7.2130035936524   -1.00964934099027⎤
⎢                                    ⎥
⎣-1.00964934099027  0.551643981070955⎦

In [58]:
- Minv @ partial(G.subs(const_list), x).subs(qbar_slist)

⎡0  -30.2473460482347  0  0⎤
⎢                          ⎥
⎣0  16.5262985013326   0  0⎦

In [61]:
Minv @ partial(Text, u).subs(slist)

⎡18.9121017496781 ⎤
⎢                 ⎥
⎣-3.59097464074083⎦